In [34]:
import kotlinx.coroutines.Dispatchers
import kotlinx.coroutines.async
import kotlinx.coroutines.awaitAll
import kotlinx.coroutines.runBlocking
import com.google.common.hash.Hashing
import io.github.cdimascio.dotenv.Dotenv
import java.nio.file.Paths

%useLatestDescriptors
%use lets-plot

class Main

val dotenv = Dotenv.load()
val dataDir = dotenv.get("DATA_DIR").let { Paths.get(it).toFile() }.also { it.mkdirs() }
dataDir

/home/j.zeidler@GDAAG.DE/private/ba3/bachelor-thesis/data

In [35]:
import org.bson.BsonDocument
import org.litote.kmongo.*

val client = KMongo.createClient("mongodb://localhost:42692/")
val db = client.getDatabase("s5_snyk_libio")
val vulnCollection = db.getCollection<BsonDocument>("mergedVuln")

In [36]:
import com.mongodb.client.*

private class MongoCursorIterable<T>(private val cursor: MongoCursor<T>) : MongoCursor<T> by cursor, Iterable<T> {

    override fun iterator(): Iterator<T> = cursor
}

private fun <T> MongoIterable<T>.kCursor(): MongoCursorIterable<T> = MongoCursorIterable(iterator())

fun <T, R> MongoIterable<T>.useCursor(block: (Iterable<T>) -> R): R {
    return kCursor().use(block)
}

In [37]:
import org.bson.BsonNull
import org.bson.BsonString
import org.jetbrains.kotlinx.dataframe.math.mean
import org.jetbrains.kotlinx.dataframe.math.median

val vulnGavToClasses = vulnCollection.find().useCursor { blk ->
    blk.map {
        val gav = it["vuln_gav"]!!.asString().value
        val vulnClasses = it["vuln_classes"]!!.asArray().map { it.asString().value }.toSet()

        if (vulnClasses.count() == 0) throw Exception("no vuln class (record should have been ommitted previously)")

        gav to vulnClasses
    }
}.groupBy { it.first }.map { g -> g.key to g.value.flatMap { it -> it.second }.toSet() }.toMap()

println(vulnGavToClasses.count())
println(vulnGavToClasses.map { it.value.size }.minOrNull())
println(vulnGavToClasses.map { it.value.size }.maxOrNull())
println(vulnGavToClasses.map { it.value.size }.average())
println(vulnGavToClasses.map { it.value.size }.median())

3809
1
51
3.549750590706222
2


In [38]:
val vulnCveGavToClasses = vulnCollection.find().useCursor { blk ->
    blk.map {
        val gav = it["vuln_gav"]!!.asString().value
        val cve = it["cve_ref"]!!.asString().value.let { c -> if (c.isBlank()) it["snyk_url"]!!.asString().value else c }
        val vulnClasses = it["vuln_classes"]!!.asArray().map { it.asString().value }.toSet()

        if (vulnClasses.count() == 0) throw Exception("no vuln class (record should have been ommitted previously)")

        (cve to gav) to vulnClasses
    }
}.groupBy { it.first }.map { g -> g.key to g.value.flatMap { it -> it.second }.toSet() }.toMap()
    
println(vulnCveGavToClasses.count())
println(vulnCveGavToClasses.map { it.value.size }.minOrNull())
println(vulnCveGavToClasses.map { it.value.size }.maxOrNull())
println(vulnCveGavToClasses.map { it.value.size }.average())
println(vulnCveGavToClasses.map { it.value.size }.median())

7618
1
51
2.0490942504594383
1


In [39]:
import scripts.partitionDepGraph.dagP.loadDagpPartitionInfo

val partitionInfo = vulnGavToClasses.keys.map { gav ->
    gav to loadDagpPartitionInfo(gav)
}.toMap()

partitionInfo.count()

3809

In [40]:
val data = mapOf(
    "partition count" to partitionInfo.values.map { it.partitionCount }
)

val p = letsPlot(data) { x = "partition count" } +
        geomHistogram(alpha = .3, binWidth = 1, center = 0.5) +
        xlim(limits = Pair(1, 33)) +
        ggsize(1800, 800) 
p

0 
 
 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 
 
 8 
 
 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 
 
 12 
 
 
 
 
 
 
 
 
 14 
 
 
 
 
 
 
 
 
 16 
 
 
 
 
 
 
 
 
 18 
 
 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 
 
 22 
 
 
 
 
 
 
 
 
 24 
 
 
 
 
 
 
 
 
 26 
 
 
 
 
 
 
 
 
 28 
 
 
 
 
 
 
 
 
 30 
 
 
 
 
 
 
 
 
 32 
 
 
 
 
 
 
 
 
 34 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 900 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 1,100 
 
 
 
 
 
 
 1,200 
 
 
 
 
 
 
 1,300 
 
 
 
 
 
 
 
 count 
 
 
 
 
 partition count

In [41]:
val depGraphCacheDir = dataDir.resolve("interim/depGraphCache")

val gavToDepGraphCache = vulnGavToClasses
    .map {
        val cachePath = depGraphCacheDir.resolve("${it.key}.vertices.tsv.zip")
        if (!cachePath.isFile()) throw Exception("cache file not found")
        
        it.key to cachePath
    }
    .toMap()
gavToDepGraphCache.count()

3809

In [42]:
import common.DefaultGraph
import io.github.classgraph.ClassInfoList
import java.io.File
import java.net.URLClassLoader
import org.jgrapht.Graph
import org.jgrapht.graph.DefaultEdge
import org.jgrapht.graph.builder.GraphTypeBuilder

fun loadDepGraph(gav: String): DefaultGraph = 
    scripts.exportDepGraphs.loadDepGraphFromCache(gav)

fun loadClassListInfo(gav: String): ClassInfoList =
    scripts.exportDepGraphs.loadClassInfoListFromCache(gav)

fun loadVertexInfo(gav: String) =
    scripts.exportDepGraphs.loadVertexInfo(gav)

In [43]:
import com.google.common.collect.Queues

/**
 * returns the depths of each visited node from the starting node
 */
fun bfsOnDepGraph(graph: DefaultGraph, startNode: String): Map<String, Int> {
    val expanded = mutableSetOf<String>()
    val depthMap = mutableMapOf<String, Int>()
    val queue = Queues.newArrayDeque<String>()
    queue.add(startNode)
    depthMap[startNode] = 0
    while (!queue.isEmpty()) {
        val node = queue.pop()
        expanded.add(node)
        val parentDepth = depthMap[node]!!
        graph
            .outgoingEdgesOf(node)
            .map { e -> graph.getEdgeTarget(e) }
            .filterNot { expanded.contains(it) }
            .forEach { v ->
                if (depthMap.containsKey(v)) {
                    depthMap[v] = min(depthMap[v]!!, parentDepth + 1)
                } else {
                    depthMap[v] = parentDepth + 1;
                }
                queue.add(v)
            }
    }
    return depthMap
}

### how many modules are vuln?

In [44]:
import kotlinx.coroutines.async
import org.jgrapht.graph.EdgeReversedGraph
import org.nield.kotlinstatistics.normalize
import org.nield.kotlinstatistics.standardDeviation
import java.lang.reflect.Modifier

data class VulnAnalysisInfo(
    val classCount: Int,
    val publicClassCount: Int,
    val vulnClassCount: Int,
    val vulnPublicClassCount: Int,
    
    val publicApiInSafeModules: Int,
    val publicApiInVulnModules: Int,
    
    val moduleSizes: List<Int>,
    val safeModuleSizes: List<Int>,
    val vulnModuleSizes: List<Int>,
) {
    val moduleCount = moduleSizes.size
    val safeModuleCount = safeModuleSizes.size
    val vulnModuleCount = vulnModuleSizes.size
    
    val publicApiInSafeModulesRatio = publicApiInSafeModules.toDouble() / publicClassCount.toDouble()
    val publicApiInVulnModulesRatio = publicApiInVulnModules.toDouble() / publicClassCount.toDouble()
    
    val moduleSizesCV = moduleSizes.standardDeviation() / moduleSizes.average()
}

val dispatcher = Dispatchers.IO.limitedParallelism(32)

@kotlinx.coroutines.ExperimentalCoroutinesApi fun _getVulnAnalysisInfo() =
    vulnCveGavToClasses.asSequence().windowed(512, 512, true).flatMapIndexed { batch, w ->
        runBlocking {
            System.gc()
            println("processing batch $batch")
            w.map {
                async(dispatcher) {
                    val gav = it.key.second
                    val vulnClasses = it.value

                    val depGraph = loadDepGraph(gav)
                    val vertexInfo = loadVertexInfo(gav)
                    val depPartitionInfo = partitionInfo[gav]!!
                    
                    if (depPartitionInfo.partToVertices.values.flatten().toSet().size != depGraph.vertexSet().size) {
                        error("failed sanity check")
                    }

                    val publicClasses =
                        depGraph.vertexSet().filter { v -> Modifier.isPublic(vertexInfo[v]!!) }

                    val distToNearestVuln = mutableMapOf<String, Int>()
                    vulnClasses.map {
                        bfsOnDepGraph(EdgeReversedGraph(depGraph), it)
                    }.forEach { m ->
                        m.forEach { v, d ->
                            if (distToNearestVuln.containsKey(v)) {
                                distToNearestVuln[v] = min(distToNearestVuln[v]!!, d)
                            } else {
                                distToNearestVuln[v] = d
                            }
                        }
                    }

                    val allVulnClasses = distToNearestVuln.keys.toSet()
                    val vulnPublicClasses = publicClasses.intersect(allVulnClasses)

                    val vulnParts =
                        depPartitionInfo.partToVertices.filter { (p, vl) ->
                            vl.intersect(allVulnClasses).isNotEmpty()
                        }
                    
                    val classesInVulnParts = vulnParts.values.flatten().toSet()
                    
                    val safeParts = 
                        depPartitionInfo.partToVertices.filterNot { (p, vl) -> 
                            vulnParts.containsKey(p)
                        }
                    
                    val classesInSafeParts = safeParts.values.flatten().toSet()
                    
                    val publicClassesInSafeParts = 
                        publicClasses.intersect(classesInSafeParts)
                    
                    val publicClassesInVulnParts =
                        publicClasses.intersect(classesInVulnParts)
                    
                    if (publicClassesInSafeParts.size + publicClassesInVulnParts.size != publicClasses.size) {
                        error("size mismatch: ${publicClassesInSafeParts.size} + ${publicClassesInVulnParts.size} != ${publicClasses.size}")
                    }
                    
                    VulnAnalysisInfo(
                        classCount = depGraph.vertexSet().size,
                        publicClassCount = publicClasses.size,
                        vulnClassCount = allVulnClasses.size,
                        vulnPublicClassCount = vulnPublicClasses.size,
                        moduleSizes = depPartitionInfo.partToVertices.map { it.value.size }.toList(),
                        safeModuleSizes = safeParts.map { it.value.size }.toList(),
                        vulnModuleSizes = vulnParts.map { it.value.size }.toList(),
                        publicApiInSafeModules = publicClassesInSafeParts.size,
                        publicApiInVulnModules = publicClassesInVulnParts.size,
                    )
                }
            }.toList().awaitAll().toList()  //.filter { it.v != null }
        }.toList()
    }.toList()

val res = _getVulnAnalysisInfo()
res.count()
// println(res.filter { it.msg != null }.map { it.msg!! }.joinToString("\n"))

processing batch 0
processing batch 1
processing batch 2
processing batch 3
processing batch 4
processing batch 5
processing batch 6
processing batch 7
processing batch 8
processing batch 9
processing batch 10
processing batch 11
processing batch 12
processing batch 13
processing batch 14


7618

In [45]:
fun Double.format(digits: Int) =
    "%.${digits}f".format(this)

fun Float.format(digits: Int) =
    "%.${digits}f".format(this)

In [46]:
println("${res.map { it.vulnModuleCount.toDouble() / it.moduleCount.toDouble() }.average()}")
println("${res.map { it.vulnModuleCount.toDouble() / it.moduleCount.toDouble() }.median()}")
println("${res.filter { it.moduleCount > 1 }.map { it.vulnModuleCount.toDouble() / it.moduleCount.toDouble() }.average()}")
println("${res.filter { it.moduleCount > 1 }.map { it.vulnModuleCount.toDouble() / it.moduleCount.toDouble() }.median()}")
println("${res.map { it.safeModuleCount.toDouble() / it.moduleCount.toDouble() }.average()}")
println("${res.map { it.safeModuleCount.toDouble() / it.moduleCount.toDouble() }.median()}")
println("${res.filter { it.moduleCount > 1 }.map { it.safeModuleCount.toDouble() / it.moduleCount.toDouble() }.average()}")
println("${res.filter { it.moduleCount > 1 }.map { it.safeModuleCount.toDouble() / it.moduleCount.toDouble() }.median()}")

0.50300890981885
0.5
0.48467699401116104
0.5
0.4969910901811499
0.5
0.515323005988839
0.5


In [47]:
println("${res.map { it.vulnModuleSizes.average() / it.classCount }.average()}")

0.2785938564396186


In [48]:
println("${res.map { it.moduleSizesCV }.median()}")
println("${res.map { it.moduleSizesCV }.average()}")

0.2668535358396509
0.45181865812655947


In [49]:
println("${res.map { it.moduleSizes.average() / it.classCount.toDouble() }.average()}")
println("${res.map { it.moduleSizes.median() / it.classCount.toDouble() }.average()}")
println("${res.map { it.moduleSizes.median() / it.classCount.toDouble() }.median()}")
println("${res.map { it.moduleSizes.average() / it.classCount.toDouble() }.median()}")

0.2549246029141507
0.22883071797789134
0.13672922252010725
0.25


In [50]:
println("${res.map { it.vulnModuleSizes.average() / it.classCount.toDouble() }.average()}")
println("${res.map { it.vulnModuleSizes.median() / it.classCount.toDouble() }.average()}")
println("${res.map { it.vulnModuleSizes.median() / it.classCount.toDouble() }.median()}")
println("${res.map { it.vulnModuleSizes.average() / it.classCount.toDouble() }.median()}")

0.2785938564396186
0.2608804585059
0.1719411395532054
0.23853211009174313


In [51]:
println("${res.map { it.publicApiInSafeModulesRatio }.average()}")
println("${res.map { it.publicApiInSafeModulesRatio }.median()}")
println("${res.map { it.publicApiInVulnModulesRatio }.average()}")
println("${res.map { it.publicApiInVulnModulesRatio }.median()}")

0.4517544345170307
0.48717948717948717
0.5482455654829694
0.5128205128205128


In [52]:
val p =
    letsPlot(
        mapOf("module size cv" to res.map { it.moduleSizesCV * 100 })
    ) { x = "module size cv" } + ggsize(1800, 800) +
            geomHistogram(alpha = .3, binWidth = 5, center = 2.5) +
            xlim(limits = Pair(0, 100))

p

-5 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 
 
 15 
 
 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 
 
 25 
 
 
 
 
 
 
 
 
 30 
 
 
 
 
 
 
 
 
 35 
 
 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 
 
 45 
 
 
 
 
 
 
 
 
 50 
 
 
 
 
 
 
 
 
 55 
 
 
 
 
 
 
 
 
 60 
 
 
 
 
 
 
 
 
 65 
 
 
 
 
 
 
 
 
 70 
 
 
 
 
 
 
 
 
 75 
 
 
 
 
 
 
 
 
 80 
 
 
 
 
 
 
 
 
 85 
 
 
 
 
 
 
 
 
 90 
 
 
 
 
 
 
 
 
 95 
 
 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 
 
 105 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 900 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 1,100 
 
 
 
 
 
 
 1,200 
 
 
 
 
 
 
 1,300 
 
 
 
 
 
 
 1,400 
 
 
 
 
 
 
 
 count 
 
 
 
 
 module size cv